In [6]:
import json
import logging

SITE = "http://ieda.ust.hk/eng/event_detail.php?type=E&id="

settings = None
try:
    with open("settings.json", "r") as f:
        settings = json.load(f)
except FileNotFoundError:
    logging.error("settings.json not found")

eventID = 803

In [7]:
import urllib.request
contents = urllib.request.urlopen(SITE+str(eventID)).read()

from bs4 import BeautifulSoup
soup = BeautifulSoup(contents, 'html.parser')

In [9]:
title = soup.find(class_='context__subtitle').text.strip()
title

''

In [3]:
soup.find_all('span', class_='item-label')


[<span class="item-label">Date</span>,
 <span class="item-label">Time</span>,
 <span class="item-label">Venue</span>,
 <span class="item-label">School</span>,
 <span class="item-label">Speaker</span>]

In [5]:
soup.find_all('span', class_='item-value')

for label, value in zip(soup.find_all('span', class_='item-label'), soup.find_all('span', class_='item-value')):
    print(label.text, value.text)

Date 4 August 2023 (Friday)
Time 10:30 a.m.
Venue Room 5562 (lift 27-28)
School Desautels Faculty of Management, McGill University
Speaker Prof. Yichuan DING , Associate Professor
